In [ ]:
# EDA enligt https://www.kaggle.com/code/ayushikaushik/eda-regression-analysis#Preprocessing-the-data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train_data = pd.read_csv('/home/paaske/Documents/ML-project/data//stores_train.csv')
busstops = pd.read_csv('/home/paaske/Documents/ML-project/data//busstops_norway.csv')
grunnkrets_age = pd.read_csv('/home/paaske/Documents/ML-project/data//grunnkrets_age_distribution.csv')
grunnkrets_households = pd.read_csv('/home/paaske/Documents/ML-project/data//grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('/home/paaske/Documents/ML-project/data//grunnkrets_income_households.csv')
grunnkrets_stripped = pd.read_csv('/home/paaske/Documents/ML-project/data//grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('/home/paaske/Documents/ML-project/data//plaace_hierarchy.csv')

print(f"Shape of training data: {train_data.shape}\nFeatures available: {train_data.columns}")

In [ ]:
# To see all the columns in output this can be done.
pd.options.display.max_columns=None
# To see all rows change max_columns with max_rows

train_data.head()

In [ ]:
# Check if there are missing values => true, so there are
train_data.isnull().sum().any()

In [ ]:
# Replace NaN in mall_name column with 'No mall'
train_data.mall_name = train_data.mall_name.fillna('No mall')
train_data.address = train_data.address.fillna('No address')
train_data.chain_name = train_data.chain_name.fillna('No chain')

In [ ]:
train_data.head()

In [ ]:
# Check if there are missing values => false, so there aren't anymore :)
train_data.isnull().sum().any()

In [ ]:
# 'store_name', 'year', 'sales_channel_name', 'address' columns are redundant, remove them
train_data = train_data.drop('store_name',axis=1)
train_data = train_data.drop('year',axis=1)
train_data = train_data.drop('sales_channel_name',axis=1)
train_data = train_data.drop('address',axis=1)

In [ ]:
#train_data['store_name'] = pd.factorize(train_data['store_name'])[0]

In [ ]:
sns.distplot(train_data['revenue'],hist=False)
plt.title('Distribution of Target variable')
sns.despine() # removes top and right border from the figure

In [ ]:
sns.distplot(np.log(train_data['revenue']),hist=False)
plt.title('Distribution of Target variable')

In [ ]:
# The data is ≈ normally distributed when plotted in log

In [ ]:
train_data.isnull().sum().sum()

In [ ]:
# Make new column for less specified plaace hierarchy to group data together
train_data = pd.merge(train_data, plaace_hierarchy[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')
train_data.isnull().sum().sum()

In [ ]:
# Check if there are missing values
train_data.isnull().sum().any() # True
train_data.update(train_data[['plaace_hierarchy_id','lv1','lv2','lv3']].fillna(0))
train_data.isnull().sum().sum()

In [ ]:
sns.countplot(x=train_data["lv2"])

In [ ]:
train_data['lv2'].value_counts()

In [ ]:
train_data.groupby('lv2')['revenue'].mean()
# hierarchy seems to affect the revenue
# Problem: why NaN on 1.5 and 3.1?

In [ ]:
train_data['lv2'].value_counts()

In [ ]:
train_data['chain_name'].value_counts()
# 307 different ones, create dummy variables will give too many columns

In [ ]:
train_data['grunnkrets_id'].value_counts()
# 3817 different ones, group together somehow? Municipality?

In [ ]:
train_data['mall_name'].value_counts()
# 488 different ones, but 10579 of the stores are not in a mall
# => make dummy variable: one column with 1 for mall and 0 for no mall

In [ ]:
# Dummy variable for mall or no mall

# This makes dummy variables for each mall with 1 and 0
# mall_dummy = pd.get_dummies(train_data['mall_name'],drop_first=True)
# train_data = pd.concat([train_data,mall_dummy],axis=1)
# train_data.drop(['mall_name'],axis=1, inplace=True)

train_data.loc[train_data['mall_name'].str.contains("No mall", na=False),'mall_dummy'] = 0
train_data.loc[~(train_data['mall_name'].str.contains("No mall", na=False)),'mall_dummy'] = 1
train_data.drop(['mall_name'],axis=1, inplace=True)


In [ ]:
# Sum together the number of people in each grunnkrets
grunnkrets_age = grunnkrets_age.drop_duplicates(subset=['grunnkrets_id'], keep='last') # if there is value for 2016 we keep it, otherwise 2015
grunnkrets_age = grunnkrets_age.fillna(0)
grunnkrets_age = grunnkrets_age.drop('year',axis=1)
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(str)
grunnkrets_age['total_nbr_people'] = grunnkrets_age.sum(axis=1) # total number of inhabitants
grunnkrets_age['group1'] = grunnkrets_age.iloc[:,1:11].sum(axis=1) # 0-9 years old
grunnkrets_age['group2'] = grunnkrets_age.iloc[:,11:21].sum(axis=1) # 10-19 years old etc
grunnkrets_age['group3'] = grunnkrets_age.iloc[:,21:31].sum(axis=1)
grunnkrets_age['group4'] = grunnkrets_age.iloc[:,31:41].sum(axis=1)
grunnkrets_age['group5'] = grunnkrets_age.iloc[:,41:51].sum(axis=1)
grunnkrets_age['group6'] = grunnkrets_age.iloc[:,51:61].sum(axis=1)
grunnkrets_age['group7'] = grunnkrets_age.iloc[:,61:71].sum(axis=1)
grunnkrets_age['group8'] = grunnkrets_age.iloc[:,71:81].sum(axis=1)
grunnkrets_age['group9'] = grunnkrets_age.iloc[:,81:92].sum(axis=1) # 80-90 years old
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(int)

train_data = pd.merge(train_data, grunnkrets_age[['grunnkrets_id', 'total_nbr_people', 'group1', 'group2', 'group3', 'group4', 'group5', 'group6', 'group7', 'group8', 'group9']], on='grunnkrets_id', how='left')
# print(train_data.isnull().sum().sum())
# Problem: We get lots of NaN

In [ ]:
# Number of people per store type (lv2) and grunnkrets

# In dataframe grunnkrets_stripped, add column for number of stores total
number_stores = train_data['grunnkrets_id'].value_counts().rename_axis('grunnkrets_id').reset_index(name='store_counts_total') # Not including NaN (stores without a grunnkrets_id)
grunnkrets_stripped = pd.merge(grunnkrets_stripped, number_stores[['grunnkrets_id', 'store_counts_total']], on='grunnkrets_id', how='left')
grunnkrets_stripped.store_counts_total = grunnkrets_stripped.store_counts_total.fillna(0)

# Different way to do the above, same result
# grunnkrets_stripped['number_stores_total'] = 0
# for i in range(len(number_stores)):
#     id = number_stores._get_value(i,'grunnkrets_id')
#     nbr = number_stores._get_value(i,'counts')
#     grunnkrets_stripped['number_stores_total'].mask(grunnkrets_stripped['grunnkrets_id'] == id, nbr, inplace=True)

# In dataframe grunnkrets_stripped, add column for number of people in the grunnkrets
grunnkrets_stripped = pd.merge(grunnkrets_stripped, grunnkrets_age[['grunnkrets_id', 'total_nbr_people']], on='grunnkrets_id', how='left')

# In dataframe grunnkrets_stripped, add column for number of people per store in total
grunnkrets_stripped['nbr_people_per_store_in_grunnkrets'] = grunnkrets_stripped['total_nbr_people']/grunnkrets_stripped['store_counts_total']

# In dataframe train_data, add column for number of people per store in total
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'nbr_people_per_store_in_grunnkrets']], on='grunnkrets_id', how='left')


# In dataframe grunnkrets_stripped, add column for number of stores in each category in lv2
# In dataframe grunnkrets_stripped, add column for number of people per store in each category in lv2
# In dataframe train_data, add column for number of people per store in each category in lv2 in each grunnkrets


In [ ]:
counts = train_data[["store_id", "grunnkrets_id", "lv2"]].groupby(
    ["grunnkrets_id", "lv2"]
).count().reset_index()
counts.columns = ["grunnkrets_id", "lv2", "counts_gr_lv2"]
counts

In [ ]:
train_data.merge(counts, how="left", on=["grunnkrets_id", "lv2"])

In [ ]:
# Number of people per area unit per grunnkrets
grunnkrets_stripped['nbr_people_per_km2'] = grunnkrets_stripped['total_nbr_people']/grunnkrets_stripped['area_km2']
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'nbr_people_per_km2']], on='grunnkrets_id', how='left')
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

In [ ]:
train_data.update(train_data[['total_nbr_people','group1','group2','group3','group4','group5','group6','group7','group8','group9']].fillna(0))
train_data.update(train_data[['nbr_people_per_store_in_grunnkrets','nbr_people_per_km2']].fillna(0))
train_data.chain_name = train_data.chain_name.fillna('No chain')
# train_data['chain_name'] = train_data['chain_name'].replace(1,'No chain')
train_data['chain_name'].value_counts()


In [ ]:
# Add municipality column to train_data
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'municipality_name']], on='grunnkrets_id', how='left')
# we get a bunch of duplicates of store_ids...? Remove them.
train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')

In [ ]:
train_data.head()

In [ ]:
# Add column with mean revenue for each chain
# Obs: mean revenue is listed for stores with No chain as well!
chains = train_data.filter(['chain_name','revenue'], axis=1)
chains = chains.groupby('chain_name').mean()
chains = chains.rename(columns={'revenue':'mean_revenue_for_chain'})
train_data.merge(chains, how="left", on=["chain_name"])

In [ ]:
# Check if there are missing values
train_data.isnull().sum().any()

In [ ]:
# GEOGRAPHICAL FEATURES

In [ ]:
# A way to get the zipcode from the lat and lon, based on https://www.geeksforgeeks.org/get-the-city-state-and-country-names-from-latitude-and-longitude-using-python/
# and https://gis.stackexchange.com/questions/352961/convert-lat-lon-to-zip-postal-code-using-python
# # requires pip install geopy in terminal first

# import geopy
# geolocator = geopy.Nominatim(user_agent="geoapiExercises")

# # FIRST ATTEMPT
# train_data.insert(len(train_data.columns),'zipcode', 0)
# # go through the whole columns of lat and lon pairs
# for i in range(train_data.shape[0]):
#     latitude = str(train_data.loc[i]['lat'])
#     longitude = str(train_data.loc[i]['lon'])
#     location = geolocator.reverse(latitude+","+longitude)
#     address = location.raw['address']
#     train_data.loc[i]['zipcode'] = address.get('postcode')
# # now we should have the dataframe with an extra column for zipcodes

# # SECOND ATTEMPT
# def get_zipcode(df, geolocator, lat_field, lon_field):
#     location = geolocator.reverse((df[lat_field], df[lon_field]))
#     return location.raw['address']['postcode']

# zipcodes = train_data.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='lat', lon_field='lon')
# train_data.insert(len(train_data.columns),'zipcode', zipcodes)
# # Now we should have the dataframe train_data with a new column for zipcodes



In [ ]:
# IDEAS/TO DO

# Convert categorical columns to numeric
# Split into train and test data set
# Make new column for distance to nearest busstop based on coordinates and busstops_norway.csv
# Make categories for income and age distribution for different grunnkretser, add as columns
# Try the sklearn ML models used in the EDA link from Kaggle